Import libraries

In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
from typing import List, Optional
import re
from __future__ import annotations
from title_cleaner import clean_job_titles
from location_cleaner import enrich_locations
from salary_parser_int import clean_salary_columns_int 

Import the dataset

In [2]:
df = pd.read_excel('all_jobs.xlsx')
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,site,job_url,job_url_direct,title,company,location,job_type,date_posted,salary_source,interval,min_amount,max_amount,currency,is_remote,job_level,job_function,company_industry,listing_type,emails,description,company_url,company_url_direct,company_addresses,company_num_employees,company_revenue,company_description,logo_photo_url,banner_photo_url,ceo_name,ceo_photo_url,mean_salary,cleaned_description
0,0,0,0,487e95701c0d55b3,indeed,https://www.indeed.com/viewjob?jk=487e95701c0d...,https://jobs.gusto.com/postings/riscpoint-advi...,Cybersecurity Operations Senior Consultant,RISCPoint,"Remote, US",fulltime,2024-10-10,direct_data,yearly,105000.0,135000.0,USD,1.0,NaN,NaN,NaN,NaN,NaN,RISCPoint is seeking a conceptual thinker with...,https://www.indeed.com/cmp/Riscpoint,https://www.riscpoint.com/,"2814 Detroit Ave \nCleveland, Ohio 44113, US",11 to 50,NaN,RISCPoint Advisory Group is an industry leader...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_heade...,NaN,NaN,120000.0,riscpoint is seeking a conceptual thinker with...
1,1,1,1,679ff0528b73610f,indeed,https://www.indeed.com/viewjob?jk=679ff0528b73...,https://www.marketaxess.com/careers/current-op...,Senior Information Security Engineer (GRC),MarketAxess Holdings,"New York, NY, US",fulltime,2024-10-10,direct_data,yearly,150000.0,200000.0,USD,0.0,NaN,NaN,NaN,NaN,NaN,Company Description\n\n\nMarketAxess is on a j...,https://www.indeed.com/cmp/Marketaxess-Holdings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175000.0,company description marketaxess is on a journe...
2,2,2,2,41f6f886cd9b9d1d,indeed,https://www.indeed.com/viewjob?jk=41f6f886cd9b...,https://workforcenow.adp.com/mascsr/default/md...,Information System Security Officer II,Global Resource Solutions,"Colorado Springs, CO, US",fulltime,2024-10-10,direct_data,yearly,84617.0,107144.0,USD,0.0,NaN,NaN,Consulting And Business Services,NaN,NaN,"Global Resource Solutions, Inc. (GRS) is seeki...",https://www.indeed.com/cmp/Global-Resource-Sol...,https://www.grsco.com/,Fairfax,51 to 200,$5M to $25M (USD),NaN,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,NaN,NaN,NaN,95880.5,"global resource solutions, inc. (grs) is seeki..."
3,3,3,3,f82f829d7cf62384,indeed,https://www.indeed.com/viewjob?jk=f82f829d7cf6...,https://workforcenow.adp.com/mascsr/default/md...,Cyber Operations Support Specialist,"Cole Engineering Services, Inc","Fort Meade, MD, US",NaN,2024-10-10,direct_data,yearly,105000.0,130000.0,USD,0.0,NaN,NaN,NaN,NaN,NaN,**Company Overview:**\n\n\nBy Light Profession...,https://www.indeed.com/cmp/Cole-Engineering-Se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117500.0,**company overview:** by light professional it...
4,4,4,4,895d9f28e9e5ee10,indeed,https://www.indeed.com/viewjob?jk=895d9f28e9e5...,https://mufgub.wd3.myworkdayjobs.com/MUFG-Care...,"Identity & Access Management, Analyst",MUFG,"Tampa, FL, US",fulltime,2024-10-10,direct_data,yearly,83000.0,109000.0,USD,1.0,NaN,NaN,NaN,NaN,NaN,**Do you want your voice heard and your action...,https://www.indeed.com/cmp/Mufg,https://careers.mufgamericas.com/,Tokyo,"10,000+",more than $10B (USD),MUFG (Mitsubishi UFJ Financial Group) is the s...,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,https://d2q79iu7y748jz.cloudfront.net/s/_heade...,Kevin Cronin,https://d2q79iu7y748jz.cloudfront.net/s/_ceoph...,96000.0,**do you want your voice heard and your action...


In [ ]:
df.head(30)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,site,job_url,job_url_direct,title,company,location,job_type,date_posted,salary_source,interval,min_amount,max_amount,currency,is_remote,job_level,job_function,company_industry,listing_type,emails,description,company_url,company_url_direct,company_addresses,company_num_employees,company_revenue,company_description,logo_photo_url,banner_photo_url,ceo_name,ceo_photo_url,mean_salary,cleaned_description,title_clean,location_missing,location_city,location_state,location_country,location_display,is_remote_display,job_type_list,job_type_primary,job_type_display,min_salary,max_salary
0,0,0,0,487e95701c0d55b3,indeed,https://www.indeed.com/viewjob?jk=487e95701c0d...,https://jobs.gusto.com/postings/riscpoint-advi...,Cybersecurity Operations Senior Consultant,RISCPoint,"Remote, US",fulltime,2024-10-10,direct_data,yearly,105000.0,135000.0,USD,True,NaN,NaN,NaN,NaN,NaN,RISCPoint is seeking a conceptual thinker with...,https://www.indeed.com/cmp/Riscpoint,https://www.riscpoint.com/,"2814 Detroit Ave \nCleveland, Ohio 44113, US",11 to 50,NaN,RISCPoint Advisory Group is an industry leader...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_heade...,NaN,NaN,120000,riscpoint is seeking a conceptual thinker with...,Other,False,Unknown,Unknown,US,Remote,Remote,[fulltime],fulltime,fulltime,120000,120000
1,1,1,1,679ff0528b73610f,indeed,https://www.indeed.com/viewjob?jk=679ff0528b73...,https://www.marketaxess.com/careers/current-op...,Senior Information Security Engineer (GRC),MarketAxess Holdings,"New York, NY, US",fulltime,2024-10-10,direct_data,yearly,150000.0,200000.0,USD,False,NaN,NaN,NaN,NaN,NaN,Company Description\n\n\nMarketAxess is on a j...,https://www.indeed.com/cmp/Marketaxess-Holdings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175000,company description marketaxess is on a journe...,Software Engineer,False,New York,NY,US,"New York, NY",On-site,[fulltime],fulltime,fulltime,175000,175000
2,2,2,2,41f6f886cd9b9d1d,indeed,https://www.indeed.com/viewjob?jk=41f6f886cd9b...,https://workforcenow.adp.com/mascsr/default/md...,Information System Security Officer II,Global Resource Solutions,"Colorado Springs, CO, US",fulltime,2024-10-10,direct_data,yearly,84617.0,107144.0,USD,False,NaN,NaN,Consulting And Business Services,NaN,NaN,"Global Resource Solutions, Inc. (GRS) is seeki...",https://www.indeed.com/cmp/Global-Resource-Sol...,https://www.grsco.com/,Fairfax,51 to 200,$5M to $25M (USD),NaN,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,NaN,NaN,NaN,95880,"global resource solutions, inc. (grs) is seeki...",Other,False,Colorado Springs,CO,US,"Colorado Springs, CO",On-site,[fulltime],fulltime,fulltime,95880,95880
3,3,3,3,f82f829d7cf62384,indeed,https://www.indeed.com/viewjob?jk=f82f829d7cf6...,https://workforcenow.adp.com/mascsr/default/md...,Cyber Operations Support Specialist,"Cole Engineering Services, Inc","Fort Meade, MD, US",NaN,2024-10-10,direct_data,yearly,105000.0,130000.0,USD,False,NaN,NaN,NaN,NaN,NaN,**Company Overview:**\n\n\nBy Light Profession...,https://www.indeed.com/cmp/Cole-Engineering-Se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117500,**company overview:** by light professional it...,Other,False,Fort Meade,MD,US,"Fort Meade, MD",On-site,None,None,Unknown,117500,117500
4,4,4,4,895d9f28e9e5ee10,indeed,https://www.indeed.com/viewjob?jk=895d9f28e9e5...,https://mufgub.wd3.myworkdayjobs.com/MUFG-Care...,"Identity & Access Management, Analyst",MUFG,"Tampa, FL, US",fulltime,2024-10-10,direct_data,yearly,83000.0,109000.0,USD,False,NaN,NaN,NaN,NaN,NaN,**Do you want your voice heard and your action...,https://www.indeed.com/cmp/Mufg,https://careers.mufgamericas.com/,Tokyo,"10,000+",more than $10B (USD),MUFG (Mitsubishi UFJ Financial Group) is the s...,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,https://d2q79iu7y748jz.cloudfront.net/s/_heade...,Kevin Cronin,https://d2q79iu7y748jz.cloudfront.net/s/_ceoph...,96000,**do you want your voice heard and your action...,Other,False,Tampa,FL,US,"Tampa, FL",On-site,[full

: 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94529 entries, 0 to 94528
Data columns (total 36 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0.2           94529 non-null  int64         
 1   Unnamed: 0.1           94529 non-null  int64         
 2   Unnamed: 0             94529 non-null  int64         
 3   id                     94529 non-null  object        
 4   site                   94529 non-null  object        
 5   job_url                94529 non-null  object        
 6   job_url_direct         60491 non-null  object        
 7   title                  94529 non-null  object        
 8   company                93220 non-null  object        
 9   location               94086 non-null  object        
 10  job_type               51639 non-null  object        
 11  date_posted            94529 non-null  datetime64[ns]
 12  salary_source          72422 non-null  object        
 13  i

In [4]:
df["title"].value_counts(dropna=False)

title
Software Engineer                                 995
Senior Software Engineer                          779
Data Scientist                                    719
Product Manager                                   508
Software Developer                                460
                                                 ... 
Software Engineer 2 (TS/SCI with FS Polygraph)      1
DevOps Engineer 2 (TS/SCI with FS Polygraph)        1
Senior Information Security Engineer (GRC)          1
Cybersecurity Operations Senior Consultant          1
Developer - Service Now                             1
Name: count, Length: 57053, dtype: int64

In [5]:
# Keep the top 10 raw non-dev/data titles as-is and label the rest "Other":
df = clean_job_titles(df, title_col="title", out_col="title_clean", coerce_other_to_sw=False, keep_top_n_other=10)

Replace the null values of the column company with the "Unknown" string


In [6]:
df['company'] = df['company'].fillna('Unknown')

Inspect the column location for potential issues and clean it accordingly

In [7]:
df["location"].value_counts(dropna=False)

location
Remote, US               4835
New York, NY, US         1979
US                       1643
Seattle, WA, US          1632
Washington, DC, US       1459
                         ... 
Desert Ridge, AZ, USA       1
Zephyrhills, FL, US         1
Anderson, CA, US            1
Ronks, PA, US               1
Moore, SC, US               1
Name: count, Length: 7715, dtype: int64

In [ ]:
enrich_locations(df, loc_col="location", fill_unknown=True)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,site,job_url,job_url_direct,title,company,location,job_type,date_posted,salary_source,interval,min_amount,max_amount,currency,is_remote,job_level,job_function,company_industry,listing_type,emails,description,company_url,company_url_direct,company_addresses,company_num_employees,company_revenue,company_description,logo_photo_url,banner_photo_url,ceo_name,ceo_photo_url,mean_salary,cleaned_description,title_clean,location_missing,location_city,location_state,location_country,location_display
0,0,0,0,487e95701c0d55b3,indeed,https://www.indeed.com/viewjob?jk=487e95701c0d...,https://jobs.gusto.com/postings/riscpoint-advi...,Cybersecurity Operations Senior Consultant,RISCPoint,"Remote, US",fulltime,2024-10-10,direct_data,yearly,105000.0,135000.0,USD,True,NaN,NaN,NaN,NaN,NaN,RISCPoint is seeking a conceptual thinker with...,https://www.indeed.com/cmp/Riscpoint,https://www.riscpoint.com/,"2814 Detroit Ave \nCleveland, Ohio 44113, US",11 to 50,NaN,RISCPoint Advisory Group is an industry leader...,NaN,https://d2q79iu7y748jz.cloudfront.net/s/_heade...,NaN,NaN,120000.0,riscpoint is seeking a conceptual thinker with...,Other,False,Unknown,Unknown,US,Remote
1,1,1,1,679ff0528b73610f,indeed,https://www.indeed.com/viewjob?jk=679ff0528b73...,https://www.marketaxess.com/careers/current-op...,Senior Information Security Engineer (GRC),MarketAxess Holdings,"New York, NY, US",fulltime,2024-10-10,direct_data,yearly,150000.0,200000.0,USD,False,NaN,NaN,NaN,NaN,NaN,Company Description\n\n\nMarketAxess is on a j...,https://www.indeed.com/cmp/Marketaxess-Holdings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,175000.0,company description marketaxess is on a journe...,Software Engineer,False,New York,NY,US,"New York, NY"
2,2,2,2,41f6f886cd9b9d1d,indeed,https://www.indeed.com/viewjob?jk=41f6f886cd9b...,https://workforcenow.adp.com/mascsr/default/md...,Information System Security Officer II,Global Resource Solutions,"Colorado Springs, CO, US",fulltime,2024-10-10,direct_data,yearly,84617.0,107144.0,USD,False,NaN,NaN,Consulting And Business Services,NaN,NaN,"Global Resource Solutions, Inc. (GRS) is seeki...",https://www.indeed.com/cmp/Global-Resource-Sol...,https://www.grsco.com/,Fairfax,51 to 200,$5M to $25M (USD),NaN,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,NaN,NaN,NaN,95880.5,"global resource solutions, inc. (grs) is seeki...",Other,False,Colorado Springs,CO,US,"Colorado Springs, CO"
3,3,3,3,f82f829d7cf62384,indeed,https://www.indeed.com/viewjob?jk=f82f829d7cf6...,https://workforcenow.adp.com/mascsr/default/md...,Cyber Operations Support Specialist,"Cole Engineering Services, Inc","Fort Meade, MD, US",NaN,2024-10-10,direct_data,yearly,105000.0,130000.0,USD,False,NaN,NaN,NaN,NaN,NaN,**Company Overview:**\n\n\nBy Light Profession...,https://www.indeed.com/cmp/Cole-Engineering-Se...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,117500.0,**company overview:** by light professional it...,Other,False,Fort Meade,MD,US,"Fort Meade, MD"
4,4,4,4,895d9f28e9e5ee10,indeed,https://www.indeed.com/viewjob?jk=895d9f28e9e5...,https://mufgub.wd3.myworkdayjobs.com/MUFG-Care...,"Identity & Access Management, Analyst",MUFG,"Tampa, FL, US",fulltime,2024-10-10,direct_data,yearly,83000.0,109000.0,USD,False,NaN,NaN,NaN,NaN,NaN,**Do you want your voice heard and your action...,https://www.indeed.com/cmp/Mufg,https://careers.mufgamericas.com/,Tokyo,"10,000+",more than $10B (USD),MUFG (Mitsubishi UFJ Financial Group) is the s...,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,https://d2q79iu7y748jz.cloudfront.net/s/_heade...,Kevin Cronin,https://d2q79iu7y748jz.cloudfront.net/s/_ceoph...,96000.0,**do you want your voice heard and your action...,Other,False,Tampa,FL,US,"Tampa, FL"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94524,94524,34350,704,5e97bb179de9eccf,indeed,https://www.indeed.com/viewjob?jk=5e97bb179de9...,https://www.t

Transform the is_remote column to a more user-friendly format

In [9]:
df['is_remote_display'] = df['is_remote'].map({True: 'Remote', False: 'On-site'}).fillna('Unknown')
# Make it categorical
df['is_remote_display'] = df['is_remote_display'].astype('category')

Inspect the column job_type for potential issues and clean it accordingly

In [10]:
df["job_type"].value_counts(dropna=False)

job_type
fulltime                                               43156
NaN                                                    42890
contract                                                4007
fulltime, contract                                      1462
parttime                                                 801
internship                                               713
parttime, fulltime                                       426
fulltime, internship                                     255
other                                                    166
temporary                                                151
temporary, fulltime                                      106
parttime, contract                                        73
temporary, contract                                       68
parttime, fulltime, contract                              49
parttime, internship                                      41
temporary, fulltime, contract                             34
parttime, fullt

In [11]:
# Priority: prefer fulltime when multiple values present,
# then contract, parttime, internship, temporary, perdiem, other, volunteer
PRIORITY = ["fulltime", "contract", "parttime", "internship", "temporary", "perdiem", "other", "volunteer"]

def normalize_token(tok: str) -> str:
    t = tok.strip().lower()
    # remove surrounding punctuation
    t = re.sub(r"^[\(\[\{'\"]+|[\)\]\}'\"]+$", "", t).strip()
    t = t.replace("-", "").replace(" ", "")
    # map common variants
    if t in ("fulltime", "fulltimejob", "fte", "permanent"):
        return "fulltime"
    if t in ("contract", "contractor", "tempcontract"):
        return "contract"
    if t in ("parttime", "parttimejob"):
        return "parttime"
    if t in ("intern", "internship", "internships"):
        return "internship"
    if t in ("temporary", "temp"):
        return "temporary"
    if t in ("perdiem", "perdiems"):
        return "perdiem"
    if t in ("volunteer",):
        return "volunteer"
    if t in ("other", "misc"):
        return "other"
    return t  # fallback to whatever normalized token remains

def parse_job_types(cell: object) -> Optional[List[str]]:
    if pd.isna(cell):
        return None
    s = str(cell)
    # remove outer parentheses and quotes, then split on common separators
    s = re.sub(r"^[\(\[\{\"\']+|[\)\]\}\"\']+$", "", s).strip()
    parts = re.split(r"[,/;|]+", s)
    tokens = [normalize_token(p) for p in parts if p and p.strip()]
    # dedupe preserving order
    seen = set()
    out = []
    for t in tokens:
        if t not in seen:
            seen.add(t)
            out.append(t)
    return out or None

def choose_primary(job_list: Optional[List[str]], priority=PRIORITY) -> Optional[str]:
    if not job_list:
        return None
    for p in priority:
        if p in job_list:
            return p
    return job_list[0]  # fallback to first token if none in priority

# apply to dataframe
df["job_type_list"] = df["job_type"].apply(parse_job_types)

# derive canonical primary job_type
df["job_type_primary"] = df["job_type_list"].apply(lambda l: choose_primary(l, PRIORITY))

# display-friendly column for dashboards/exports (no nulls)
df["job_type_display"] = df["job_type_primary"].fillna("Unknown").astype("category")

Replace null values of the columns min_salary and max_salary with the median of each column


In [12]:
df = clean_salary_columns_int(df,min_col="min_amount", max_col="max_amount", mean_col="mean_salary", overwrite=True, fill_strategy="median", enforce_int=True)

Save new cleaned dataset as a CSV file with selected columns

In [13]:
selected_columns = ["title_clean", "company", "location_city", "location_state", "location_country", "job_type_display", "is_remote_display", "min_salary", "max_salary", "mean_salary"]
df[selected_columns].to_csv("./output/cleaned_all_jobs.csv", index=False)